In [1]:
from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt
import pandas as pd

In [2]:
df_annoter = pd.read_csv("../0data/resultats_catego_unique_annoter.csv", sep=';')
df_annoter.shape

(4982, 13)

In [3]:
df_without_stopwords = pd.read_csv("../../resultats_categorisation_without_stopwords.csv")

df_merge_cat = df_annoter[['id', 'level_1_annoter', 'level_2_annoter', 
'level_3_annoter', 'level_4_annoter']].merge(df_without_stopwords, on='id')

In [3]:
df_with_stopwords = pd.read_csv("../../result_catego_organicResult_knowledgeGraph_withoutCleaning_withoutCleaning.csv")

df_merge_cat = df_annoter[['id', 'level_1_annoter', 'level_2_annoter', 
'level_3_annoter', 'level_4_annoter']].merge(df_with_stopwords, on='id')

In [4]:
df_merge_cat.shape

(4982, 17)

In [5]:
df_merge_cat = df_merge_cat.drop(columns=['Unnamed: 0.1', 'Unnamed: 0', 
'dates_reference'])

In [6]:
df_merge_cat.isna().sum()

id                    0
level_1_annoter     435
level_2_annoter      84
level_3_annoter     518
level_4_annoter    3465
level_1               0
level_2              81
level_3              81
level_4            3754
model                 0
clean_scrapping       0
account_id            0
amount                0
descriptions          0
dtype: int64

In [7]:
df_an = df_merge_cat.copy()

In [8]:
df_an = df_an.dropna(subset=['level_1_annoter'])
df_an.shape

(4547, 14)

In [9]:
df_an.isna().sum()

id                    0
level_1_annoter       0
level_2_annoter      79
level_3_annoter      83
level_4_annoter    3030
level_1               0
level_2              81
level_3              81
level_4            3334
model                 0
clean_scrapping       0
account_id            0
amount                0
descriptions          0
dtype: int64

In [10]:
df_an = df_an.fillna('')

In [11]:
df_an.isna().sum()

id                 0
level_1_annoter    0
level_2_annoter    0
level_3_annoter    0
level_4_annoter    0
level_1            0
level_2            0
level_3            0
level_4            0
model              0
clean_scrapping    0
account_id         0
amount             0
descriptions       0
dtype: int64

In [12]:
df_an = df_an[~(df_an.level_2_annoter=='???')]
df_an.shape

(4546, 14)

In [13]:
df_ml = df_an[df_an.model=='ML']
df_rules = df_an[df_an.model=='rules']

In [14]:
df_ml.level_2_annoter.value_counts()

level_2_annoter
food                     433
leisure_activity         282
other                    241
financial_institution    120
transport                 95
health                    45
charge                    30
Name: count, dtype: int64

In [15]:
df_ml.level_2.value_counts()

level_2
other                    786
leisure_activity         172
food                     170
transport                 50
health                    29
financial_institution     21
charge                    18
Name: count, dtype: int64

In [16]:
df_an.model.value_counts(True)*100

model
rules    72.591289
ML       27.408711
Name: proportion, dtype: float64

In [17]:
from PIL import Image, ImageDraw, ImageFont


def classification_report_image(report, path):
    # Create an image
    image = Image.new("RGB", (500, 1000), "white")

    # Create a drawing context
    draw = ImageDraw.Draw(image)

    # Define font and font size
    font = ImageFont.load_default()

    # Split the classification report into lines
    lines = report.split("\n")

    # Set the starting y-coordinate for the text
    y = 10

    # Write each line of the report to the image
    for line in lines:
        draw.text((10, y), line, fill="black", font=font)
        y += 20

    # Save the image as a PNG file
    image.save(path)

In [18]:
import numpy as np

def plot_confusion_matrix(conf_matrix, class_names, height, width, title, fontsize=12):
    plt.figure(figsize=(height, width))
    plt.imshow(conf_matrix, interpolation='nearest', cmap=plt.get_cmap('Blues'))
    plt.title('Confusion Matrix')
    plt.colorbar()

    tick_marks = np.arange(len(class_names))
    plt.xticks(tick_marks, class_names, rotation=45)
    plt.yticks(tick_marks, class_names)

    for i in range(len(class_names)):
        for j in range(len(class_names)):
            plt.text(j, i, str(conf_matrix[i, j]), horizontalalignment="center", fontsize=fontsize, color="black")

    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    plt.xticks(rotation=90)
    plt.tight_layout()

    # Display the plot or save it as an image
    # plt.show()
    # To save as an image (e.g., PNG):
    plt.savefig(title, format='png')


In [19]:
def result(dataframe, level, height=10, width=10, title='foo'):
    dict_level_value = {
        'level_1': ['autres', 'cb', 'depot', 'reimbursement', 'retrait', 'sepa', 'vir'], 
        'level_2': ['', 'charge', 'financial_institution', 'food', 'health', 'income',
        'leisure_activity', 'other', 'transport'], 
        'level_3': ['', 'A categoriser', 'account_mouvement', 'animal', 'automobile', 'catering', 'cheque_deposit', 'consumer_loan', 'cultural_fun_electronic goods', 'e_shop', 'family', 'fee', 'fine', 'gambling', 'insurance', 'konbini', 'large_retailer', 'layout', 'medical', 'other', 'other_market', 'other_revenue', 'outfits', 'outing', 'pharmacy', 'physical', 'plane', 'public_transport', 'reimbursement', 'rent', 'salary', 'social_aid', 'streaming', 'taxe', 'taxi', 'telecom', 'utilities', 'withdrawal'], 
        'level_4': ['', 'bakery_pastry_shop', 'butchery', 'culture', 'delivery', 'e_shop', 'fast_food', 'fuel', 'hypermarket_supermarket', 'layout', 'other', 'outing', 'parking', 'proxi_shop', 'restaurant', 'spare_parts', 'toll']
    }

    y_true = dataframe[level+'_annoter'].tolist()
    y_pred = dataframe[level].tolist()

    cr = classification_report(y_true, y_pred)#, target_names=dict_level_value[level])
    classification_report_image(cr, path='image/cr_'+title+'.png')

    #cm = confusion_matrix(y_true, y_pred)#, labels=dict_level_value[level])

    #disp = ConfusionMatrixDisplay(
    #    confusion_matrix=cm, 
    #    display_labels=dict_level_value[level])

    #fig, ax = plt.subplots(figsize=(height, width))
    #disp.plot(ax=ax)
    #plt.xticks(rotation=90)
    #fig.savefig('image/'+title+'.png')
    #plt.close(fig)

    #plot_confusion_matrix(cm, dict_level_value[level], height, width, fontsize=12, title='image/'+title+'.png')


In [20]:
df_rules.level_3_annoter.value_counts()

level_3_annoter
large_retailer                   458
A categoriser                    323
withdrawal                       216
catering                         212
consumer_loan                    167
cultural_fun_electronic goods    163
telecom                          132
outfits                          125
automobile                       120
insurance                        115
other                            109
salary                           104
social_aid                        99
other_revenue                     98
                                  79
fee                               72
reimbursement                     71
account_mouvement                 70
pharmacy                          68
public_transport                  67
other_market                      62
gambling                          58
konbini                           54
streaming                         45
family                            43
utilities                         40
cheque_deposit        

In [36]:
result(df_an, 'level_1', height=5, width=5, title='organicResult_knowledgeGraph_withoutCleaningl1')

In [37]:
result(df_an, 'level_2', height=5, width=5, title='organicResult_knowledgeGraph_withoutCleaningl2')

In [38]:
result(df_an, 'level_3', height=10, width=10, title='organicResult_knowledgeGraph_withoutCleaningl3')

/anaconda/envs/azureml_py310_sdkv2/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/anaconda/envs/azureml_py310_sdkv2/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/anaconda/envs/azureml_py310_sdkv2/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(r

In [39]:
result(df_an, 'level_4', height=10, width=10, title='organicResult_knowledgeGraph_withoutCleaningl4')

/anaconda/envs/azureml_py310_sdkv2/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/anaconda/envs/azureml_py310_sdkv2/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/anaconda/envs/azureml_py310_sdkv2/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(r

In [40]:
result(df_rules, 'level_2', height=5, width=5, title='organicResult_knowledgeGraph_withoutCleaning_rules_l2')

In [41]:
result(df_rules, 'level_3', height=5, width=5, title='organicResult_knowledgeGraph_withoutCleaning_rules_l3')

In [42]:
result(df_rules, 'level_4', height=5, width=5, title='organicResult_knowledgeGraph_withoutCleaning_rules_l4')

In [43]:
#df_ml[(df_ml.level_2_annoter=='other') & (df_ml.level_3_annoter=='other')]


In [44]:
result(df_ml, 'level_2', height=5, width=5, title='organicResult_knowledgeGraph_withoutCleaning_ml_l2')

In [45]:
result(df_ml, 'level_3', height=10, width=10, title='organicResult_knowledgeGraph_withoutCleaning_ml_l3')

/anaconda/envs/azureml_py310_sdkv2/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/anaconda/envs/azureml_py310_sdkv2/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/anaconda/envs/azureml_py310_sdkv2/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(r

In [46]:
result(df_ml, 'level_4', height=10, width=10, title='organicResult_knowledgeGraph_withoutCleaning_ml_l4')

/anaconda/envs/azureml_py310_sdkv2/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/anaconda/envs/azureml_py310_sdkv2/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/anaconda/envs/azureml_py310_sdkv2/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(r

In [32]:
df_ml['same'] = df_ml.apply(lambda x: x.level_2_annoter==x.level_2, axis=1)
df_ml['tuple'] = df_ml.apply(lambda x: (x.level_2, x.level_2_annoter), axis=1)
df_ml[df_ml.same==False].tuple.value_counts()


/tmp/ipykernel_48889/1641224944.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_ml['same'] = df_ml.apply(lambda x: x.level_2_annoter==x.level_2, axis=1)
/tmp/ipykernel_48889/1641224944.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_ml['tuple'] = df_ml.apply(lambda x: (x.level_2, x.level_2_annoter), axis=1)


tuple
(other, food)                                261
(other, leisure_activity)                    140
(other, financial_institution)                93
(other, transport)                            44
(other, health)                               24
(leisure_activity, food)                      19
(other, charge)                               16
(leisure_activity, other)                     14
(food, other)                                  9
(food, leisure_activity)                       9
(financial_institution, other)                 4
(leisure_activity, financial_institution)      4
(health, leisure_activity)                     3
(leisure_activity, transport)                  3
(health, financial_institution)                3
(health, other)                                3
(leisure_activity, health)                     3
(transport, other)                             3
(charge, food)                                 2
(transport, leisure_activity)                  2
(food, transpo

In [47]:
import shutil
shutil.make_archive(
    "result_catego_organicResult_knowledgeGraph_withoutCleaning", 
    'zip', 
    'image')


'/mnt/batch/tasks/shared/LS_root/mounts/clusters/yash1/code/Users/yash/categorisation/annotation_accuracy/result_catego_organicResult_knowledgeGraph_withoutCleaning.zip'